In [1]:
from v3_pool import *

## Test constructors

In [2]:
Token("ETH",18)

Token(name='ETH', decimals=18)

In [3]:
GlobalState()

GlobalState(L=0.0, rP=0.0, tick=0, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)

In [4]:
TickState()

TickState(liq_net=0.0, liq_gross=0.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)

In [5]:
PositionState()

PositionState(liq=0.0, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)

In [6]:
Pool("ETH", 18, "USDC", 6, bootstrap_rP=4000**0.5)

ETH-USDC pool - tick spacing 1
Token(name='ETH', decimals=18) Token(name='USDC', decimals=6)
GlobalState(L=0.0, rP=63.24351886516985, tick=82944, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserve X=0.0 Y=0.0
cumulative HMM X_adj=0.0 Y_adj=0.0


## Test static method formulas

Problem 1:  
A user has x = 2 ETH and wants to set up a liquidity position in an ETH/USDC pool. The current price of ETH is P =2000 USDC and target price range is from pa = 1500 to pb = 2500 USDC. How much USDC (y) do they need?

In [7]:
# 2-step: get L first
a,P,b = 1500,2000,2500
x = 2.0 
# First, calculate the liquidity of the top half of the range by using
# reserve x cover from current P to upper bound (pretend P is lower bound)
Lx = Pool.liq_x_only(x=x,rPa=P**0.5, rPb=b**0.5)
Lx

847.2135954999583

In [8]:
# 2-step: then get y . 
y = Pool.y_from_L_rP_rng(L=Lx,rP=P**0.5,rPa=a**0.5,rPb=b**0.5)
y

5076.102359479882

Problem 2:  
A user has x=2 ETH and y=4000 USDC, and wants to use pb =3000USDC perETH as the top of the price range. What is the bottom of the range (pa) that ensures the opened position uses the full amount of their funds?

In [9]:
# Bounds in 1-step
x,y  = 2.0, 4000
P = y/x
b= 3000

rPa = Pool.rPa_from_x_y_rP_rPb(x=x,y=y,rP=P**0.5,rPb=b**0.5)
rPa**2

1333.3333333333333

In [10]:
# Bounds in 2-steps
# compute L using the top part of range, given we know x 
Lx = Pool.liq_x_only(x,rPa=P**0.5,rPb=b**0.5)
# then get lower bound from L, price and y
rPa = Pool.rPa_from_L_rP_y(L=Lx,rP=P**0.5,y=y)
rPa**2

1333.3333333333337

Problem 3:  
Using the liquidity position created in Problem 2, what are asset balances when the price changes to P = 2500 USDC per ETH?

In [11]:
# compute L from price when position established, reserves and bounds
L = Pool.liq_from_x_y_rP_rng(x=x,y=y,rP=P**0.5, rPa=rPa, rPb=b**0.5)

x_new = Pool.x_from_L_rP_rng(L=L, rP=2500**0.5, rPa=rPa, rPb=b**0.5)

y_new = Pool.y_from_L_rP_rng(L=L, rP=2500**0.5, rPa=rPa, rPb=b**0.5)

x_new , y_new

(0.8493641204744684, 6572.9000439693455)

In [12]:
# or calc change of reserves based on change of price
dX =  Pool.dX_from_L_drP(L,rP_old=P**0.5,rP_new=2500**0.5)
print(f"{L=} {dX=} x_new={x + dX}")

L=487.41718030204123 dX=-1.1506358795255318 x_new=0.8493641204744682


In [13]:
dY =  Pool.dY_from_L_drP(L,rP_old=P**0.5,rP_new=2500**0.5)
print(f"{dY=} y_new={y + dY}")

dY=2572.9000439693473 y_new=6572.900043969347


-------------
-------------
-------------
## Test pool mechanics

### Pool Init and tick math

In [14]:
pool= Pool("ETH", 18, "USDC", 6, bootstrap_rP=4000**0.5,tick_spacing=30)
pool

ETH-USDC pool - tick spacing 30
Token(name='ETH', decimals=18) Token(name='USDC', decimals=6)
GlobalState(L=0.0, rP=63.16767594946435, tick=82920, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserve X=0.0 Y=0.0
cumulative HMM X_adj=0.0 Y_adj=0.0


In [15]:
assert(pool.tick_to_rP(pool.global_state.tick) <= pool.global_state.rP)

In [16]:
print(pool.rP_to_tick(pool.global_state.rP), pool.rP_to_possible_tick(pool.global_state.rP))
print(pool.rP_to_tick(pool.global_state.rP), pool.rP_to_possible_tick(pool.global_state.rP, left_to_right=True))

82920 82920
82920 82950


------------
------------
1. set_position()

In [17]:
pool= Pool("ETH", 18, "USDC", 6, bootstrap_rP=4000**0.5,tick_spacing=100)
pool

ETH-USDC pool - tick spacing 100
Token(name='ETH', decimals=18) Token(name='USDC', decimals=6)
GlobalState(L=0.0, rP=63.1045430018443, tick=82900, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserve X=0.0 Y=0.0
cumulative HMM X_adj=0.0 Y_adj=0.0


In [18]:

pool._set_position("123", 80000, 90000, 300)
pool.show()


GlobalState(L=300.0, rP=63.1045430018443, tick=82900, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=0.0 Y=0.0
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '80000': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '90000': TickState(liq_net=-300.0, liq_gross=300.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('123', 80000, 90000)': PositionState(liq=300, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


In [19]:
pool._set_position("888", 90000,95000 , 200)
pool.show()


GlobalState(L=300.0, rP=63.1045430018443, tick=82900, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=0.0 Y=0.0
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '80000': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '90000': TickState(liq_net=-100.0, liq_gross=500.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '95000': TickState(liq_net=-200.0, liq_gross=200.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('123', 80000, 90000)': PositionState(liq=300, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)
poz '('888', 90000, 95000)': PositionState(liq=200, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


In [20]:
pool._set_position("888", 70000, 95000, 400)
pool.show()


GlobalState(L=700.0, rP=63.1045430018443, tick=82900, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=0.0 Y=0.0
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '80000': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '90000': TickState(liq_net=-100.0, liq_gross=500.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '95000': TickState(liq_net=-600.0, liq_gross=600.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '70000': TickState(liq_net=400.0, liq_gross=400.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
---positions---
poz '('123', 80000, 90000)': PositionState(liq=300, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)
poz '('888', 90000, 95000)': PositionState(liq=200, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)
poz '('888', 70000, 95000)': PositionState(liq=400, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


In [21]:
pool._set_position("888", 90000, 95000, -200)
pool.show()


GlobalState(L=700.0, rP=63.1045430018443, tick=82900, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=0.0 Y=0.0
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '80000': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '90000': TickState(liq_net=-300.0, liq_gross=300.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '95000': TickState(liq_net=-400.0, liq_gross=400.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '70000': TickState(liq_net=400.0, liq_gross=400.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
---positions---
poz '('123', 80000, 90000)': PositionState(liq=300, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)
poz '('888', 70000, 95000)': PositionState(liq=400, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


-----------
-----------
2. deposit()

In [22]:
from v3_pool import *

pool= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)


x_debited=1.9984356160534436 y_debited=3998.319535406821
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.0015643839465564469 Y returned 1.68046459317884


-----------
-----------
3. execute_swap_from_X(): breaking up trades

In [23]:
from v3_pool import *

pool1= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool1.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)

pool1.execute_swap_from_X(3)
print()
pool1.show()


x_debited=1.9984356160534436 y_debited=3998.319535406821
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.0015643839465564469 Y returned 1.68046459317884
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=2.4489254304363857 swpd_dY=-3998.3194954236255 pool_X=4.44736104648983 pool_Y=3.99831956201524e-05 avg_P=1632.683235565546, end_P=1332.9373
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0


GlobalState(L=0.0, rP=36.509413246066714, tick=71954, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=4.44736104648983 Y=3.99831956201524e-05
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '71955': TickState(liq_net=487.0, liq_gross=487.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '80067': TickState(liq_net=-487.0, liq_gross=487.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('abc', 71955, 80067)': PositionState(liq=487, fr_x=0.0, fr_y=

In [24]:
pool2= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool2.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)
pool2.execute_swap_from_X(1)
pool2.execute_swap_from_X(1)
pool2.execute_swap_from_X(1)
print()
pool2.show()
# assert pool2.X >= pool1.X, "breaking up trade should be slighlty more advantageous for pool"
# assert pool2.Y >= pool1.Y, "breaking down the trade should result in less Y going out"


x_debited=1.9984356160534436 y_debited=3998.319535406821
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.0015643839465564469 Y returned 1.68046459317884
swpd_dX=1.0 swpd_dY=-1831.6417148754679 pool_X=2.9984356160534436 pool_Y=2166.677820531353 avg_P=1831.6417148754679, end_P=1677.5941
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0
swpd_dX=1.0 swpd_dY=-1547.448158157445 pool_X=3.9984356160534436 pool_Y=619.229662373908 avg_P=1547.448158157445, end_P=1427.3988
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=0.44892543043638644 swpd_dY=-619.2296223907127 pool_X=4.44736104648983 pool_Y=3.998319527909189e-05 avg_P=1379.3596450724096, end_P=1332.9373
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0


GlobalState(L=0.0, rP=36.509413246066714, tick=71954, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)

In [25]:
pool1.execute_swap_from_X(0.75)
pool2.execute_swap_from_X(0.75)

Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=0.0 swpd_dY=0.0 pool_X=4.44736104648983 pool_Y=3.99831956201524e-05 avg_P=None, end_P=1332.9373
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=0.0 swpd_dY=0.0 pool_X=4.44736104648983 pool_Y=3.998319527909189e-05 avg_P=None, end_P=1332.9373
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0


(0.0, 0.0, 0.0, 0.0, None, 1332.9372555720715)

----------
4. execute_swap_from_X(): pool starts with no liquidity in range

In [26]:
from v3_pool import *
pool3= Pool("SOL", 12, "USDC", 6, bootstrap_rP=150**0.5,tick_spacing=1)
pool3.deposit("dude",x=4, y=600, rPa=100**0.5, rPb=140**0.5)
pool3.show()
assert pool3.X == 0.0, "no X deposited as range below price"


x_debited=0.0 y_debited=598.9437669860457
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 4.0 Y returned 1.0562330139542837

GlobalState(L=0.0, rP=12.246923203116042, tick=50108, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=0.0 Y=598.9437669860457
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '46054': TickState(liq_net=327.0, liq_gross=327.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '49418': TickState(liq_net=-327.0, liq_gross=327.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
---positions---
poz '('dude', 46054, 49418)': PositionState(liq=327, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


In [27]:
pool3.execute_swap_from_X(2)
pool3.show()

Gap in liquidity... trying to get in range...
swpd_dX=2.0 swpd_dY=-261.08182855073886 pool_X=2.0 pool_Y=337.86193843530685 avg_P=130.54091427536943, end_P=121.7318
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0

GlobalState(L=327.0, rP=11.033214725935037, tick=48020, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=2.0 Y=337.86193843530685
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '46054': TickState(liq_net=327.0, liq_gross=327.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '49418': TickState(liq_net=-327.0, liq_gross=327.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
---positions---
poz '('dude', 46054, 49418)': PositionState(liq=327, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


----------
5. execute_swap_from_X(): there is gap of liquidity on the way down while trying to fill whole amount

In [28]:
from v3_pool import *
pool4= Pool("SOL", 12, "USDC", 6, bootstrap_rP=200**0.5,tick_spacing=1)
pool4.deposit("dude",x=4, y=600, rPa=175**0.5, rPb=250**0.5)
pool4.deposit("mate",x=0, y=400, rPa=120**0.5, rPb=150**0.5)
# gap of liquidity between 175 and 150 
pool4.show()



x_debited=3.9947723653231546 y_debited=488.50299634424977
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.005227634676845394 Y returned 111.49700365575023

x_debited=0.0 y_debited=399.4265195744154
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.0 Y returned 0.5734804255845916

GlobalState(L=535.0, rP=14.141553871453628, tick=52985, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=3.9947723653231546 Y=887.9295159186652
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '51650': TickState(liq_net=535.0, liq_gross=535.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '55217': TickState(liq_net=-535.0, liq_gross=535.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '47877': TickState(liq_net=309.0, liq_gross=309.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '50108': TickState(liq_net=-309.0, liq_gross=309.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
---positions---
poz '('dude', 51650, 55217)': PositionState(liq=535, fr_x

In [29]:
pool4.execute_swap_from_X(7)
pool4.show()

Gap in liquidity... trying to get in range...
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=5.588647047527747 swpd_dY=-887.92950703937 pool_X=9.583419412850901 pool_Y=8.879295194219594e-06 avg_P=158.8809419324779, end_P=119.9963
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0

GlobalState(L=0.0, rP=10.954280744946413, tick=47876, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=9.583419412850901 Y=8.879295194219594e-06
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '51650': TickState(liq_net=535.0, liq_gross=535.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '55217': TickState(liq_net=-535.0, liq_gross=535.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '47877': TickState(liq_net=309.0, liq_gross=309.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '50108': TickState(liq_net=-309.0, liq_gross=309.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
---positions-

-----------
-----------
6. execute_swap_from_Y(): breaking up trades

In [30]:
from v3_pool import *

pool5= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool5.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)
pool5.show()
print()
pool5.execute_swap_from_Y(6000)
print()
pool5.show()


x_debited=1.9984356160534436 y_debited=3998.319535406821
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.0015643839465564469 Y returned 1.68046459317884

GlobalState(L=487.0, rP=44.71951496148113, tick=76012, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=1.9984356160534436 Y=3998.319535406821
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '71955': TickState(liq_net=487.0, liq_gross=487.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '80067': TickState(liq_net=-487.0, liq_gross=487.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('abc', 71955, 80067)': PositionState(liq=487, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)

Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=-1.998435596069088 swpd_dY=4894.779514882709 pool_X=1.9984355503765983e-08 pool_Y=8893.09905028953 avg_P=2449.305609102797, end_P=2999.7964
adjusted_dX=0.0  pool_cumul_X_adj=0.0 total_fee_Y=0

In [31]:
pool6= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool6.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)
pool6.execute_swap_from_Y(2000)
pool6.execute_swap_from_Y(2000)
pool6.execute_swap_from_Y(2000)
print()
pool6.show()
# assert pool6.Y >= pool5.Y, "breaking up trade should be slighlty more advantageous for pool"
# assert pool6.X >= pool5.X, "breaking down the trade should result in less X going out"


x_debited=1.9984356160534436 y_debited=3998.319535406821
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.0015643839465564469 Y returned 1.68046459317884
swpd_dX=-0.9159656144500636 swpd_dY=2000.0 pool_X=1.08247000160338 pool_Y=5998.319535406821 avg_P=2183.4880790811994, end_P=2384.0067
adjusted_dX=0.0  pool_cumul_X_adj=0.0 total_fee_Y=0.0  pool_cumul_Y_fee=0.0
swpd_dX=-0.7738364706837699 swpd_dY=2000.0 pool_X=0.30863353091961 pool_Y=7998.319535406821 avg_P=2584.5253820006433, end_P=2801.9096
adjusted_dX=0.0  pool_cumul_X_adj=0.0 total_fee_Y=0.0  pool_cumul_Y_fee=0.0
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=-0.30863351093525443 swpd_dY=894.7795148827092 pool_X=1.9984355559277134e-08 pool_Y=8893.099050289531 avg_P=2899.165136576557, end_P=2999.7964
adjusted_dX=0.0  pool_cumul_X_adj=0.0 total_fee_Y=0.0  pool_cumul_Y_fee=0.0


GlobalState(L=0.0, rP=54.77039692222591, tick=80067, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y

In [32]:
pool5.execute_swap_from_Y(500)
pool6.execute_swap_from_Y(500)

Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=0.0 swpd_dY=0.0 pool_X=1.9984355503765983e-08 pool_Y=8893.09905028953 avg_P=None, end_P=2999.7964
adjusted_dX=0.0  pool_cumul_X_adj=0.0 total_fee_Y=0.0  pool_cumul_Y_fee=0.0
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=0.0 swpd_dY=0.0 pool_X=1.9984355559277134e-08 pool_Y=8893.099050289531 avg_P=None, end_P=2999.7964
adjusted_dX=0.0  pool_cumul_X_adj=0.0 total_fee_Y=0.0  pool_cumul_Y_fee=0.0


(0.0, 0.0, 0.0, 0.0, None, 2999.796379018174)

----------
7. execute_swap_from_Y(): pool starts with no liquidity in range

In [33]:
from v3_pool import *
pool7= Pool("SOL", 12, "USDC", 6, bootstrap_rP=150**0.5,tick_spacing=1)
pool7.deposit("dude",x=4, y=600, rPa=180**0.5, rPb=230**0.5)
pool7.show()
assert pool7.Y == 0.0, "no Y deposited as range below price"


x_debited=3.997362440038902 y_debited=0.0
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.002637559961097935 Y returned 600.0

GlobalState(L=0.0, rP=12.246923203116042, tick=50108, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=3.997362440038902 Y=0.0
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '51932': TickState(liq_net=465.0, liq_gross=465.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '54383': TickState(liq_net=-465.0, liq_gross=465.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('dude', 51932, 54383)': PositionState(liq=465, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


In [34]:
pool7.execute_swap_from_Y(200)
pool7.show()

Gap in liquidity... trying to get in range...
swpd_dX=-1.076614681316411 swpd_dY=200.0 pool_X=2.920747758722491 pool_Y=200.0 avg_P=185.76748345606214, end_P=191.7229
adjusted_dX=0.0  pool_cumul_X_adj=0.0 total_fee_Y=0.0  pool_cumul_Y_fee=0.0

GlobalState(L=465.0, rP=13.846404750028977, tick=52563, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=2.920747758722491 Y=200.0
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '51932': TickState(liq_net=465.0, liq_gross=465.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '54383': TickState(liq_net=-465.0, liq_gross=465.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('dude', 51932, 54383)': PositionState(liq=465, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


----------
8. execute_swap_from_Y(): there is gap of liquidity on the way up while trying to fill whole amount

In [35]:
from v3_pool import *
pool8= Pool("SOL", 12, "USDC", 6, bootstrap_rP=175**0.5,tick_spacing=1)
pool8.deposit("dude",x=2, y=600, rPa=150**0.5, rPb=190**0.5)
pool8.deposit("mate",x=3, y=0, rPa=210**0.5, rPb=250**0.5)
# gap of liquidity between 190 and 210 
pool8.show()



x_debited=1.859765349010167 y_debited=599.7215238377478
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.1402346509898329 Y returned 0.2784761622522183

x_debited=2.9964567099858472 y_debited=0.0
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.003543290014152767 Y returned 0.0

GlobalState(L=611.0, rP=13.228464158660637, tick=51650, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=4.856222058996014 Y=599.7215238377478
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '50108': TickState(liq_net=611.0, liq_gross=611.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '52472': TickState(liq_net=-611.0, liq_gross=611.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '53473': TickState(liq_net=520.0, liq_gross=520.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
tick '55217': TickState(liq_net=-520.0, liq_gross=520.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('dude', 50108, 52472)': PositionState(liq=611, fr_x=0.0, fr_y=0.0, hr

In [36]:
pool8.execute_swap_from_Y(1500)
pool8.show()

Gap in liquidity... trying to get in range...
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=-4.856222010433798 swpd_dY=1025.6343846036552 pool_X=4.85622169144051e-08 pool_Y=1625.3559084414028 avg_P=211.2000609527399, end_P=249.9908
adjusted_dX=0.0  pool_cumul_X_adj=0.0 total_fee_Y=0.0  pool_cumul_Y_fee=0.0

GlobalState(L=0.0, rP=15.811095915727572, tick=55217, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=4.85622169144051e-08 Y=1625.3559084414028
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '50108': TickState(liq_net=611.0, liq_gross=611.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '52472': TickState(liq_net=-611.0, liq_gross=611.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '53473': TickState(liq_net=520.0, liq_gross=520.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '55217': TickState(liq_net=-520.0, liq_gross=520.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
--

-------------
-------------
9. withdraw()

In [37]:
from v3_pool import *
pool_w= Pool("SOL", 12, "USDC", 6, bootstrap_rP=200**0.5,tick_spacing=1)
pool_w.deposit("dude",x=4, y=600, rPa=175**0.5, rPb=250**0.5)
pool_w.show()


x_debited=3.9947723653231546 y_debited=488.50299634424977
including fees_x+adj_x=0.0 and fees_y+adj_y=0.0
X returned 0.005227634676845394 Y returned 111.49700365575023

GlobalState(L=535.0, rP=14.141553871453628, tick=52985, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=3.9947723653231546 Y=488.50299634424977
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '51650': TickState(liq_net=535.0, liq_gross=535.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '55217': TickState(liq_net=-535.0, liq_gross=535.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('dude', 51650, 55217)': PositionState(liq=535, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


In [38]:
pool_w.execute_swap_from_X(20)
# pool_w.execute_swap_from_Y(380)
pool_w.show()

Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swpd_dX=2.6113234852730938 swpd_dY=-488.5029914592198 pool_X=6.606095850596248 pool_Y=4.885029966317234e-06 avg_P=187.07103666558257, end_P=174.9923
adjusted_dY=0.0  pool_cumul_Y_adj=0.0 total_fee_X=0.0  pool_cumul_X_fee=0.0

GlobalState(L=0.0, rP=13.228464158660637, tick=51649, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=6.606095850596248 Y=4.885029966317234e-06
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
tick '51650': TickState(liq_net=535.0, liq_gross=535.0, f0_x=0.0, f0_y=0.0, h0_x=0.0, h0_y=0.0)
tick '55217': TickState(liq_net=-535.0, liq_gross=535.0, f0_x=0, f0_y=0, h0_x=0, h0_y=0)
---positions---
poz '('dude', 51650, 55217)': PositionState(liq=535, fr_x=0.0, fr_y=0.0, hr_x=0.0, hr_y=0.0)


In [39]:

pool_w.withdraw("dude",liq=535, rPa=175**0.5, rPb=250**0.5)
pool_w.show()


x_sent=6.606095850596249 y_sent=0.0
including fees_x+adj_x=0.0 & fees_y+adj_y=0.0

GlobalState(L=0.0, rP=13.228464158660637, tick=51649, fg_x=0.0, fg_y=0.0, hg_x=0.0, hg_y=0.0)
real reserves X=-8.881784197001252e-16 Y=4.885029966317234e-06
cumulative HMM X_adj=0.0 Y_adj=0.0
cumulative swap fees_X=0.0 fees_Y=0.0
---active ticks---
---positions---
